In [4]:
import { SamplingMixins, causalNetSampling } from 'causal-net.sampling';
import { Platform } from 'causal-net.utils';
import * as Core from 'causal-net.core';
var { Tensor, Functor, causalNetCore } = Core;
console.log(causalNetSampling instanceof Functor);
var T = causalNetCore.CoreTensor;

false


http://mathfaculty.fullerton.edu/mathews/n2003/QRMethodMod.html

In [5]:
var m = T.tensor([ [1, 3, 4], 
                   [3, 1, 2], 
                   [4, 2, 1]] );
function Eignh(m){
    const [s,_] = m.shape;
    diag = T.eye(s);
    var [q, r] = T.linalg.qr(m);
    var  cQ = q.dot(diag);
    let err = q.pow(2).sum().sub(q.mul(diag).pow(2).sum());
    while( err.dataSync()[0] > 1e-4 ){
        [q, r] = T.linalg.qr(m);
        err = q.pow(2).sum().sub(q.mul(diag).pow(2).sum());
        m = r.dot(q);
        cQ = cQ.dot(q);
    }
    return [m.mul(diag).sum(1), cQ]
}

In [82]:
var [s, v] = Eignh(m)
s.print()
v.print()

Tensor
    [[1, 0, 0],
     [0, 1, 0],
     [0, 0, 1]]
Tensor
    [7.0746336, -3.1878407, -0.8867912]
Tensor
    [[0.4545127 , 0.4893836, -0.7442594],
     [-0.0466633, 0.847486 , 0.5287629 ],
     [-0.8895173, 0.2055998, -0.4080295]]


In [73]:
det(v)

0.9934492984509811

In [60]:
q.gather(0)

Tensor {
  kept: false,
  isDisposedInternal: false,
  shape: [ 3 ],
  dtype: 'float32',
  size: 3,
  strides: [],
  dataId: {},
  id: 86220,
  rankType: '1',
  scopeId: 54094 }

In [ ]:
//calculate the determinant of a matrix m

function det(m) {
    return T.tidy(() => {
        let [r, _] = m.shape
        
        if (r === 2) {
            const t = m.as1D()
            const a = t.slice([0], [1]).dataSync()[0]
            const b = t.slice([1], [1]).dataSync()[0]
            const c = t.slice([2], [1]).dataSync()[0]
            const d = t.slice([3], [1]).dataSync()[0]
            result = a * d - b * c
            return result

        } else {
            let s = 0;
            
            for (let i = 0; i < r; i++) {
                rows = [...Array(r).keys()]
                sub_m = m.gather(T.tensor1d(rows.filter(e => e !== i), 'int32'))
                sli = sub_m.slice([0, 1], [r - 1, r - 1])
                _d = det(sli)
                s += Math.pow(-1, i) * _d 
            }
            return s
        }
    })
}

// the inverse of the matrix : matrix adjoint method
function invertMatrix(m) {
    return T.tidy(() => {
        const d = det(m)
        if (d === 0) {
            return
        }
        let [r, _] = m.shape
        let rows = [...Array(r).keys()]
        let dets = []
        for (let i = 0; i < r; i++) {
            for (let j = 0; j < r; j++) {
                const sub_m = m.gather(T.tensor1d(rows.filter(e => e !== i), 'int32'))
                let sli
                if (j === 0) {
                    sli = sub_m.slice([0, 1], [r - 1, r - 1])
                } else if (j === r - 1) {
                    sli = sub_m.slice([0, 0], [r - 1, r - 1])
                } else {
                    const [a, b, c] = T.split(sub_m, [j, 1, r - (j + 1)], 1)
                    sli = T.concat([a, c], 1)
                }
                dets.push(Math.pow(-1, (i + j)) * det(sli))
            }
        }
        com = T.tensor2d(dets, [r, r])
        tr_com = com.transpose()
        inv_m = tr_com.div(T.scalar(d))
        
        return inv_m
    })
}
var sh = 18
var a = T.tensor2d([...Array(sh*sh).keys()], [sh, sh])
console.log("matrix a")
a.print()
console.log("determinant")
var da = det(a)
console.log({da})
var i = invertMatrix(a)
console.log("inverse i")
i.print()
var prod = i.dot(a)
console.log("a * i")
prod.print()

matrix a
Tensor
    [[0  , 1  , 2  , 3  , 4  , 5  , 6  , 7  , 8  , 9  , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 ],
     [18 , 19 , 20 , 21 , 22 , 23 , 24 , 25 , 26 , 27 , 28 , 29 , 30 , 31 , 32 , 33 , 34 , 35 ],
     [36 , 37 , 38 , 39 , 40 , 41 , 42 , 43 , 44 , 45 , 46 , 47 , 48 , 49 , 50 , 51 , 52 , 53 ],
     [54 , 55 , 56 , 57 , 58 , 59 , 60 , 61 , 62 , 63 , 64 , 65 , 66 , 67 , 68 , 69 , 70 , 71 ],
     [72 , 73 , 74 , 75 , 76 , 77 , 78 , 79 , 80 , 81 , 82 , 83 , 84 , 85 , 86 , 87 , 88 , 89 ],
     [90 , 91 , 92 , 93 , 94 , 95 , 96 , 97 , 98 , 99 , 100, 101, 102, 103, 104, 105, 106, 107],
     [108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125],
     [126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143],
     [144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161],
     [162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179],
     [180, 181

In [33]:
[...Array(4).keys()]

[ 0, 1, 2, 3 ]